<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/071_Clean_AI_Tools_with_Dependency_Injection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 🧼 Clean AI Tools with Dependency Injection

## Why This Pattern Matters

By using `ActionContext`, we solve several key challenges in AI tool and agent architecture:

* ✅ Tools can access **conversation history** without being tightly coupled to the agent implementation
* 🔐 **Authentication** and other request-specific info can be injected where needed
* 🧪 Tools remain **independent and testable**, thanks to explicit dependency declarations
* 🔄 Agents can provide **different contexts** for development, testing, or production environments

---

## 💡 What Is `ActionContext`?

We’ve already been using `ActionContext` throughout our tools, but we haven’t talked about why it's so important. It's a powerful architectural pattern that enables **dependency injection** and **decouples tools** from the core agent logic.

---

## 🧠 Real-World Example: Code Development & Review Agent

Imagine building an AI agent that:

1. First acts as a developer to write code based on requirements
2. Later acts as a reviewer to critique its own work

Seems simple, right? But here's the architectural catch:

* For a code review to be effective, the LLM must understand the **entire context** of how the code was created:

  * What requirements were discussed?
  * What constraints were raised?
  * What tradeoffs were considered?

That history is in the **agent memory** — so how do we give the tool access to it **without tightly coupling** it to the agent itself?

---

## 🤯 The Problem Without ActionContext

Here’s a naive version of the tool:

```python
@register_tool(description="Analyze code quality and suggest improvements")
def analyze_code_quality(code: str) -> str:
    return prompt_expert(
        description_of_expert="Senior software architect reviewing code quality",
        prompt=f"Review this code:\n{code}"
    )
```

The problem?
🔗 This tool has **no access to memory** — so it can’t consider **how** the code was developed, only **what** it looks like now.

---

## 🧩 The Solution: Injecting Memory via `ActionContext`

We fix this with dependency injection:

```python
@register_tool(description="Analyze code quality and suggest improvements")
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    memory = action_context.get_memory()

    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            development_context.append(f"User: {mem['content']}")
        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            development_context.append(f"Implementation Decision: {mem['content']}")

    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1. Does the implementation meet all stated requirements?
2. Are all constraints and considerations from the discussion addressed?
3. Have any requirements or constraints been overlooked?
4. What improvements could make the code better while staying within the discussed parameters?
"""

    generate_response = action_context.get("llm")
    return generate_response(review_prompt)
```

---

✅ Now this tool is **context-aware**
✅ But still **modular and testable**
✅ And can be used **across agents, pipelines, or environments**




In [ ]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(code: str) -> str:
    """Review code quality and suggest improvements."""
    # But how do we access the conversation history?
    # We can't just import the agent instance - that would create tight coupling

    return prompt_expert(
        description_of_expert="""
        Senior software architect reviewing code quality
        """,
        prompt=f"Review this code:\n{code}"
    )



### 🧩 **The General Workflow**

#### 1. **User Input**

The user initiates the process:

> *"Please write a function that parses CSV data and handles malformed rows gracefully."*

#### 2. **Code Generation Agent**

The agent:

* Reads the user input.
* Writes an initial implementation.
* Saves this to memory along with reasoning and user requirements.
* Returns the **first draft** of the code to the user.

#### 3. **Review Trigger**

At this point:

* Either the agent or user can trigger the **code review** tool (e.g., `analyze_code_quality`).
* The tool reads memory (requirements + code) and uses an LLM to generate **feedback**:

  * Missed requirements
  * Logic bugs
  * Suggestions for improvement

#### 4. **Result Options**

The agent/system might now:

* ✅ Return **only** the review (for user to manually revise).
* 🔁 Automatically attempt a **revised version** using feedback.
* 🧠 Present **both** the review and the revision for user review.

#### 5. **User Review & Iteration**

The user might then:

* Accept the revision.
* Add clarification (e.g., *“Actually, can it also validate column types?”*)
* Request further changes.
* Ask the agent to **re-run the review** with the new context.

---

### 🔁 **Feedback Loop Summary**

```text
User → Agent → Code
             ↘
           Review Agent
             ↘
           Feedback → Revised Code
                     ↘
                  User Feedback
                        ↘
                 Restart or Accept
```

---

### 🧠 Why This Matters

This pattern:

* Mirrors how humans work: draft → review → revise → approval.
* Lets agents stay modular (creator vs reviewer).
* Enables **safe, testable, traceable** development cycles.
* Builds trust by **exposing rationale**, not just results.




### 🧠 What the Tool Is Doing

When the tool performs a **code review**, it wants to **simulate how a senior engineer would think**:

> *"Before I judge this code, I need to understand what the requirements were and how the solution came about."*

So it:

1. **Pulls memory** from `action_context` — this includes the full conversation history (user prompts, LLM responses, etc.).

2. **Filters** those memories to:

   * ✅ Capture the **user’s original instructions**, like:

     > "Write a function that parses a CSV and summarizes data by region."
   * ✅ Capture relevant **LLM responses**, such as:

     > "Here’s the implementation that does what you asked."

3. **Combines**:

   * The **intent** from the user
   * The **decisions made by the LLM**
   * The **final code snippet**

4. **Constructs a prompt** that gives the LLM this full development history for a **context-rich review**.

---

### 📊 Why This Matters

Without this memory injection:

* The LLM would only see the **final code**, not the *why* behind it.
* That’s like asking someone to critique a building without telling them what kind of building it’s supposed to be.

---

### ✅ Summary

> **Yes — you're absolutely right.**
> It captures both the **intent** (user input) and the **response** (generated code) so the final review can assess whether the code **meets the original goals**, **respects constraints**, and **offers room for improvement**.




### 🧩 What Does "Tightly Coupled" Mean?

When two components are **tightly coupled**, it means:

* One **directly depends** on the inner workings of the other
* If you change one, you’re **forced to change the other**
* They **can’t function independently**

---

### 📦 In the Agent Context

Let’s say you have a tool like:

```python
def review_code():
    memory = my_agent.memory
    llm = my_agent.llm
    ...
```

That tool is now **tightly coupled** to `my_agent` because:

* It assumes the existence of a specific agent object
* It pulls memory and LLM references directly from that agent
* You **can't reuse** this tool in another context or test it easily

If the agent changes its structure or API, this tool **breaks**.

---

### 🛠 Why This Is a Problem

* ❌ You **can’t reuse** tools across agents
* ❌ You **can’t unit test** tools in isolation
* ❌ You **can’t swap** agents or LLMs easily
* ❌ It violates the **Separation of Concerns** principle

---

### ✅ What ActionContext Enables

By injecting dependencies like this:

```python
def review_code(action_context: ActionContext):
    memory = action_context.get_memory()
    llm = action_context.get("llm")
```

You get:

* ✔ **Loose coupling** (no hardcoded references)
* ✔ **Testability** (inject mocks or stubs)
* ✔ **Portability** (use the same tool in different agents or environments)
* ✔ **Encapsulation** (tools don’t care *who* the agent is, only that they get the data they need)

---

### 🧠 Analogy

> Tightly coupled = You wrote a tool that only works in *your* kitchen
> Loosely coupled = You wrote a tool that works in *any* kitchen, as long as there’s a fridge and a stove






## 🧰 `action_context`: Decoupling Tools from Agents

We face an immediate problem:
Our tool needs access to the **conversation history** stored in memory — but we can’t simply import the agent instance or directly reference its memory.

### ❌ Why That’s a Problem

Directly accessing agent memory would:

* **Tightly couple** the tool to a specific agent implementation
* Make tools **harder to reuse** across different agents
* Complicate **testing** and **modularity**

---

### ✅ The Solution: `ActionContext` Pattern

The `ActionContext` pattern solves this by acting as a **dependency injection container** — it passes in only what the tool needs to function.

> Think of it as a backpack the agent gives to the tool, containing just the right supplies (like memory, LLM, APIs, etc.)

This allows:

* Tools to access memory **without knowing where it came from**
* Tools to remain **independent, testable, and portable**
* A clean architecture where **tools don’t import agents or global state**

---

### 🧠 Why It Matters

`ActionContext` creates a separation between:

* **Business logic** (what the tool does)
* **Execution environment** (what memory/LLM/resources are available)

This makes tools safer, more robust, and easy to scale across multiple agent workflows.





## 🔍 Understanding the `ActionContext` Class

```python
class ActionContext:
    def __init__(self, properties: Dict = None):
        self.context_id = str(uuid.uuid4())
        self.properties = properties or {}

    def get(self, key: str, default=None):
        return self.properties.get(key, default)

    def get_memory(self):
        return self.properties.get("memory", None)
```

---

### 🧱 What's Happening Here?

| Line / Concept | What It Does                                                                  | Why It Matters                                                                                                    |
| -------------- | ----------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `__init__`     | Creates a new `ActionContext` with a unique ID and a dictionary of resources. | This allows any tool to be passed context-specific data (e.g., memory, LLM, tokens, APIs) without tight coupling. |
| `context_id`   | Unique ID for the context session.                                            | Helpful for tracking/debugging tool runs in complex systems.                                                      |
| `properties`   | A flexible dictionary of all available injected dependencies.                 | Keeps things modular. Add what you need, ignore what you don’t.                                                   |
| `get(key)`     | General-purpose accessor for context data.                                    | Use this when you want to access something like `"llm"`, `"api_key"`, etc.                                        |
| `get_memory()` | Specialized helper to retrieve the memory object.                             | Frequently used by tools that need to read past conversation history.                                             |

---

### 📦 What Kind of Things Go Into `properties`?

You might inject any of the following:

* `memory`: An object that holds conversation history
* `llm`: A function that wraps LLM calls (like `generate_response(prompt)`)
* `agent_registry`: Used for agent-to-agent calls
* `api_keys`: Tokens for calendar, email, etc.
* `user_id`, `session_id`, etc.

---

### ✅ Benefits Recap

* **Testable:** Tools don’t need global state
* **Reusable:** Use the same tool across agents/systems
* **Flexible:** Context can be tailored per environment (e.g., dev vs. prod)
* **Safe:** Tools don't reach outside their scope



This second half of the code introduces **the real power of `ActionContext`** — and there are a few *key architectural principles and design patterns* you should focus on. Let’s break them down:

---

## 🔍 What You Should Focus On

### 1. **The Role of `ActionContext`: Dependency Injection**

```python
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
```

* Instead of accessing global objects, the tool receives **everything it needs** through `action_context`.
* ✅ This makes the tool **reusable, testable, and decoupled** from the environment it's running in.

---

### 2. **Accessing Memory Through Context (Not Agent)**

```python
memory = action_context.get_memory()
```

* This is **clean dependency injection**.
* You’re not importing or hardcoding any `Agent` object — the memory could come from *any* source (mock, live agent, simulated test).
* 📌 This solves the tight coupling problem.

---

### 3. **Selective Memory Parsing for Relevant Context**

```python
for mem in memory.get_memories():
    if mem["type"] == "user":
        ...
    elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
        ...
```

* You’re building a **development history context** based on conversation types.
* This lets the tool **reconstruct the narrative** of the code’s origin: what was asked, what was built.

---

### 4. **Context-Aware Prompt Engineering**

```python
review_prompt = f"""Review this code in the context of its development history:
...
"""
```

* Instead of just dumping code into the LLM, you provide **historical context** that helps it reason more effectively.
* 🧠 You’re helping the model understand *why* the code was written the way it was — just like a human reviewer would.

---

### 5. **LLM Access Through Context (Not Direct Call)**

```python
generate_response = action_context.get("llm")
return generate_response(review_prompt)
```

* Again, you’re not hardcoding `openai.ChatCompletion.create()` or anything vendor-specific.
* This makes it easy to swap in another LLM, mock it for testing, or route it through a review system.

---

## 🧠 What You're Learning

* **Architectural clean code**: Tools should be dumb. Context should be smart.
* **Tool portability**: This review tool could now be dropped into *any* codebase with any memory and LLM backend.
* **Scalability**: This pattern supports plugin systems, multi-agent systems, and runtime configuration.

---

## ✅ Why This Matters

| Principle                  | Why It Matters                             |
| -------------------------- | ------------------------------------------ |
| **Loose coupling**         | Easier to reuse and modify tools           |
| **Separation of concerns** | Tools focus only on logic, not environment |
| **Testability**            | You can unit test tools in isolation       |
| **Context awareness**      | LLMs perform better with relevant context  |
| **Vendor agnostic**        | Works with OpenAI, Anthropic, etc.         |




In [ ]:
@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    """Review code quality and suggest improvements."""
    # Get memory to understand the code's context
    memory = action_context.get_memory()

    # Extract relevant history
    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            development_context.append(f"User: {mem['content']}")
        # Hypotethical scenario where our agent includes the phrase "Here's the implementation" when it generates code
        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            development_context.append(f"Implementation Decision: {mem['content']}")

    # Create review prompt with full context
    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1. Does the implementation meet all stated requirements?
2. Are all constraints and considerations from the discussion addressed?
3. Have any requirements or constraints been overlooked?
4. What improvements could make the code better while staying within the discussed parameters?
"""

    generate_response = action_context.get("llm")
    return generate_response(review_prompt)




This is a great example of how clean, decoupled tool design can empower complex AI behaviors. Let’s break it down step by step and highlight what you should focus on and **why** it matters.

---

## 🧠 Key Purpose of This Tool

The `analyze_code_quality` tool:

* **Reviews code intelligently**, based not just on the code itself but also on the **conversation history** (requirements, decisions, trade-offs).
* Demonstrates how to use the `ActionContext` to **inject dependencies** cleanly (like memory and the LLM function).
* Makes the tool testable, reusable, and modular.

---

## 🔍 What to Focus On and Why

| Code Block                                      | 🔍 What to Focus On                                                            | 💡 Why It Matters                                                                                                                                  |
| ----------------------------------------------- | ------------------------------------------------------------------------------ | -------------------------------------------------------------------------------------------------------------------------------------------------- |
| `action_context.get_memory()`                   | Memory access is abstracted through the context.                               | Tool doesn’t need to know how or where memory is stored — it’s injected. This is key to **decoupling**.                                            |
| `memory.get_memories()`                         | Retrieves the full conversation history.                                       | Enables the tool to reconstruct the **development process** that led to the current code.                                                          |
| Conditional logic: `if mem["type"] == ...`      | Selectively extracts relevant history (user input, assistant code generation). | Reduces token usage and improves relevance — don’t dump the whole memory, just the important parts.                                                |
| `"Here's the implementation" in mem["content"]` | Using a **heuristic** to identify code-generation messages.                    | Demonstrates lightweight, human-readable methods to tag or extract agent actions.                                                                  |
| `review_prompt = f"""..."""`                    | Builds a custom LLM prompt using **dynamic context**.                          | This is where the tool becomes smart. It combines memory and current input to produce an informed, context-aware review request.                   |
| `generate_response = action_context.get("llm")` | LLM is injected, not hardcoded.                                                | You could swap in a different model (e.g., GPT-4 vs Claude) just by changing the context — great for testing or different deployment environments. |
| `return generate_response(review_prompt)`       | Tool calls the LLM with a structured, scoped task.                             | Clean separation: **tool defines behavior**, **LLM executes it**. No entanglement with the tool internals.                                         |

---

## 🧩 Summary: What You're Learning

This tool teaches you:

✅ How to use memory to provide intelligent context
✅ How to build clean, testable tools with **dependency injection**
✅ How to combine current input + past conversation to produce better LLM prompts
✅ How to keep tools **modular** without losing power or flexibility

This is *exactly* the kind of design that scales well, remains safe, and keeps agents reliable.




In [ ]:
#===============================
# V1: Naive version (Tightly Coupled)
#===============================

@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(code: str) -> str:
    """Review code quality and suggest improvements."""

    # ❗️Problem: This version lacks context about how the code was written
    # It assumes that code alone is enough, but good review needs to understand requirements, goals, and trade-offs

    # ❗️Also: No memory or external resources can be injected — this makes it hard to test or adapt

    return prompt_expert(
        description_of_expert="""
        Senior software architect reviewing code quality
        """,
        prompt=f"Review this code:\n{code}"
    )


In [ ]:
#===============================
# ActionContext Utility Class
#===============================

class ActionContext:
    def __init__(self, properties: Dict=None):
        self.context_id = str(uuid.uuid4())  # Unique ID for traceability/debugging
        self.properties = properties or {}   # Store injected dependencies or state

    def get(self, key: str, default=None):
        # Generic getter — flexible and extendable
        return self.properties.get(key, default)

    def get_memory(self):
        # Memory accessor — useful abstraction
        return self.properties.get("memory", None)

#===============================
# V2: Context-Aware Tool (Decoupled)
#===============================

@register_tool(
    description="Analyze code quality and suggest improvements",
    tags=["code_quality"]
)
def analyze_code_quality(action_context: ActionContext, code: str) -> str:
    """Review code quality and suggest improvements."""

    # ✅ Dependency injection: Tool does not assume where memory comes from
    memory = action_context.get_memory()

    # Extract relevant development history from memory
    development_context = []
    for mem in memory.get_memories():
        if mem["type"] == "user":
            # Capture user input (e.g. requirements or constraints)
            development_context.append(f"User: {mem['content']}")

        elif mem["type"] == "assistant" and "Here's the implementation" in mem["content"]:
            # Heuristic: Capture key assistant responses that signal code generation
            development_context.append(f"Implementation Decision: {mem['content']}")

    # 🧠 We are building a prompt that includes both the code and the *story* of how it was created
    review_prompt = f"""Review this code in the context of its development history:

Development History:
{'\n'.join(development_context)}

Current Implementation:
{code}

Analyze:
1. Does the implementation meet all stated requirements?
2. Are all constraints and considerations from the discussion addressed?
3. Have any requirements or constraints been overlooked?
4. What improvements could make the code better while staying within the discussed parameters?
"""

    # Injected LLM function — you could swap models just by changing the context
    generate_response = action_context.get("llm")

    # Return the LLM's analysis
    return generate_response(review_prompt)


## ✅ Summary of What You Should Learn from This

### 🔧 Architectural Takeaways:

* **Decouple tools from agents** using `ActionContext`
* **Inject dependencies** instead of importing or assuming them
* **Enable testing and reuse** by keeping tools stateless and parameterized

### 🤖 AI-Specific Insights:

* Context matters — LLMs perform better with structured, relevant memory
* Design prompts to include history, decisions, and intent — not just code
* Use heuristics (like `"Here's the implementation"`) to extract key memory entries

### 🧪 Why This Scales:

This design makes your tools:

* Easier to test
* More maintainable
* Safer in production
* More adaptable across different agents or environments